In [119]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Ignore warnings about future changes in library and deprecated functions
import warnings
warnings.filterwarnings('ignore')

In [120]:
# Load the .csv data into a Pandas DataFrame with the index column set to the column with the coin_id
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display first ten rows of sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [121]:
# Generate summary statistics for the DataFrame
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [122]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width = 800,
    height = 400,
    rot = 90 # Rotates the x-axis labels to be vertical
) 

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [123]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
crypto_scaled = StandardScaler().fit_transform(df_market_data) # normalizes means to 0 and standard deviation to 1
print(crypto_scaled[0:5]) 

[[ 0.50852937  0.49319307  0.77220043  0.23545963 -0.0674951  -0.35595348
  -0.25163688]
 [ 0.18544589  0.93444504  0.55869212 -0.05434093 -0.27348273 -0.11575947
  -0.19935211]
 [ 0.02177396 -0.70633685 -0.02168042 -0.06103015  0.00800452 -0.55024692
  -0.28206051]
 [-0.04076438 -0.81092807  0.24945797 -0.05038797 -0.37316402 -0.45825882
  -0.29554614]
 [ 1.19303608  2.00095907  1.76061001  0.54584206 -0.29120287 -0.49984776
  -0.27031695]]


In [125]:
# Create a DataFrame with the scaled data and the corresponding columns
crypto_transformed = pd.DataFrame(crypto_scaled, columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d", 'price_change_percentage_30d', 'price_change_percentage_60d', 'price_change_percentage_200d',	'price_change_percentage_1y'])

# Copy the crypto names from the original data 
coins = list(df_market_data.index) # this returns a list of all the coin names from the original data

# Set the coin_id column as index
crypto_transformed["coin_id"] = coins  # create a new column in the transformed df containing the list of coins
crypto_transformed = crypto_transformed.set_index("coin_id") # set the df index to the new coin_id column

# Display first ten rows of transformed data
crypto_transformed.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351


---

### Find the Best Value for k Using the Original Data.

In [126]:
# Create a list with the number of k-values from 1 to 11
k1 = list(range(1,12)) # 12 is not included

In [127]:
# Create an empty list to store the inertia values
inertia_list1 = []


# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i1 in k1:
    k_model = KMeans(n_clusters = i1, random_state = 1) # random_state is the seed for the random number generator
    k_model.fit(crypto_transformed) # with reference to the instructions, `crypto_transformed` is essentially `df_market_data_scaled`
    inertia_list1.append(k_model.inertia_) # inertia is the sum of squared distances of samples to their closest cluster center

In [128]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k": k1, "inertia": inertia_list1}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df1 = pd.DataFrame(elbow_data)

In [129]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df1.hvplot.line(
    x = "k", 
    y = "inertia", 
    title = "Elbow Curve",
    xticks = k1
)

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for `k` = **4**. _After that point, the line becomes mostly horizontal, because more clusters only reduce the within-cluster sum of squares (WCSS) by a small amount._

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [130]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4, random_state = 1) # random_state is the seed value

In [131]:
# Fit the K-Means model using the scaled data
model.fit(crypto_transformed) # .fit is the training step

KMeans(n_clusters=4, random_state=1)

In [134]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k_4_clusters1 = model.predict(crypto_transformed) # .predict is the testing step

# Print the resulting array of cluster values.
k_4_clusters1

array([3, 3, 1, 1, 3, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3,
       1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 0, 3, 1, 1, 2, 1, 1, 1, 1])

In [135]:
# Create a copy of the DataFrame
clusters_predicted = crypto_transformed.copy()

In [136]:
# Add a new column to the DataFrame with the predicted clusters
clusters_predicted["predicted_cluster"] = k_4_clusters1 

# Display first ten rows of predicted clusters
clusters_predicted.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,3
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,3
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,1
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,1
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,3
binancecoin,0.891871,1.327295,0.800214,-0.057148,0.778653,-0.188232,-0.225533,3
chainlink,0.011397,2.572251,1.101647,-0.490495,-0.931954,0.387759,-0.018284,3
cardano,0.102530,1.508001,0.648885,0.328959,-0.486349,0.065080,-0.155428,3
litecoin,0.077497,0.334297,0.858520,-0.012646,-0.366477,-0.486266,-0.292351,3


In [137]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clusters_predicted.hvplot.scatter(
    x = "price_change_percentage_24h",
    y = "price_change_percentage_7d",
    by = "predicted_cluster",
    hover_cols = "coin_id" # this is the column that will be displayed when you hover over the data points
)

:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [138]:
# Create a PCA model instance and set `n_components=3`.
pca_instance = PCA(n_components = 3)

In [139]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
clusters_pca = pca_instance.fit_transform(clusters_predicted)

# View the first five rows of the DataFrame. 
clusters_pca[:5]

array([[-1.58513942,  0.13439444,  0.545621  ],
       [-1.32258066,  0.05449393,  1.11570854],
       [ 0.26633536, -0.53640937, -0.79694588],
       [ 0.25722513, -0.59201286, -0.6465287 ],
       [-2.95224985,  0.42237155,  1.26789353]])

In [140]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca_instance.explained_variance_ratio_

array([0.37005408, 0.32322221, 0.19115222])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the dataset when all dimensional variance is reduced to three principal components is _0.37005408 + 0.32322221 + 0.19115222_ = **0.88442851**, or just over 88%. In other words, _even after PCA reduces dimensionality, it still captures a large amount of the original dataset's variance._

In [142]:
# Create a new DataFrame with the PCA data.
cluster_pca_df = pd.DataFrame(
    clusters_pca,
    columns = ["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
cluster_pca_df["coin_id"] = list(clusters_predicted.index)

# Set the coin_id column as index
cluster_pca_df = cluster_pca_df.set_index("coin_id")

# Display first ten rows of PCA data
cluster_pca_df.head(10)

,PC1,PC2,PC3
coin_id,,,
bitcoin,-1.585139,0.134394,0.545621
ethereum,-1.322581,0.054494,1.115709
tether,0.266335,-0.536409,-0.796946
ripple,0.257225,-0.592013,-0.646529
bitcoin-cash,-2.952250,0.422372,1.267894
binancecoin,-1.975484,0.528219,0.670145
chainlink,-1.671906,0.178423,2.718598
cardano,-1.539740,0.324517,1.497226
litecoin,-1.347533,-0.132837,0.791063


---

### Find the Best Value for k Using the PCA Data

In [143]:
# Create a list with the number of k-values from 1 to 11
k2 = list(range(1,12)) # 12 is not included

In [144]:
# Create an empty list to store the inertia values
inertia_list2 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i2 in k2:
    k_model = KMeans(n_clusters = i2, random_state = 1)
    k_model.fit(cluster_pca_df) # with reference to the instructions, `cluster_pca_df` is essentially `df_market_data_pca`
    inertia_list2.append(k_model.inertia_)

In [145]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data2 = {"k": k2, "inertia": inertia_list2}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df2 = pd.DataFrame(elbow_data2)

In [146]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df2.hvplot.line(
    x = "k", 
    y = "inertia", 
    title = "Elbow Curve", 
    xticks = k2
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `k` = **4**. _After that point, the line becomes mostly horizontal, because more clusters only reduce the within-cluster sum of squares (WCSS) by a small amount._


* **Question:** Does it differ from the best `k` value found using the original data?

  * **Answer:** No, the optimal value for `k` in both cases = **4**. This consistency in the optimal `k` value between the original scaled data and the PCA-reduced data provides additional confidence in the clustering. It suggests that the underlying structure of the data – in terms of how the data points group together – is maintained _even after the dimensionality reduction of PCA_ (i.e., "throwing away" some data).


### Cluster Cryptocurrencies with K-means Using the PCA Data

In [147]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters = 4, random_state = 1) # random_state is the seed value

In [148]:
# Fit the K-Means model using the PCA data
model.fit(cluster_pca_df) # .fit is the training step

KMeans(n_clusters=4, random_state=1)

In [149]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k_4_clusters2 = model.predict(cluster_pca_df)

# Print the resulting array of cluster values.
k_4_clusters2

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 3, 0, 0, 0, 0])

In [150]:
# Create a copy of the DataFrame with the PCA data
copy_cluster_pca_df = cluster_pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
copy_cluster_pca_df["predicted_cluster"] = k_4_clusters2

# Display first ten rows of predicted clusters
copy_cluster_pca_df.head(10)

,PC1,PC2,PC3,predicted_cluster
coin_id,,,,
bitcoin,-1.585139,0.134394,0.545621,1
ethereum,-1.322581,0.054494,1.115709,1
tether,0.266335,-0.536409,-0.796946,0
ripple,0.257225,-0.592013,-0.646529,0
bitcoin-cash,-2.952250,0.422372,1.267894,1
binancecoin,-1.975484,0.528219,0.670145,1
chainlink,-1.671906,0.178423,2.718598,1
cardano,-1.539740,0.324517,1.497226,1
litecoin,-1.347533,-0.132837,0.791063,1


In [151]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
copy_cluster_pca_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = 'predicted_cluster',
    hover_cols='coin_id' # this is the column that will be displayed when you hover over the data points
)

:NdOverlay   [predicted_cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [152]:
# Composite plot to contrast the Elbow curves
elbow_df1.hvplot.line(x = "k", y = "inertia", title = "Elbow Curve 1", xticks = k1) + elbow_df2.hvplot.line(x = "k", y = "inertia", title= "Elbow Curve 2", xticks = k2)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [153]:
# Composite plot to contrast the clusters
clusters_predicted.hvplot.scatter( x = "price_change_percentage_24h", y = "price_change_percentage_7d", by = "predicted_cluster", hover_cols = "coin_id") + copy_cluster_pca_df.hvplot.scatter(x = "PC1", y = "PC2", by = 'predicted_cluster', hover_cols = 'coin_id')

:Layout
   .NdOverlay.I  :NdOverlay   [predicted_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of using PCA data results in tighter, more compact, less dispersed clusters. This is because the goal of PCA is to _capture as much of the variance in the data with fewer features (i.e., principal components)_. This can help clarify patterns in the data, especially when visualizing it. With less "noise", clusters are more condensed.